<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/aware/fashion_mnist_aware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 485.4 MB 32 kB/s 
     |████████████████████████████████| 2.3 MB 33.8 MB/s 
     |████████████████████████████████| 463 kB 43.6 MB/s 
     |████████████████████████████████| 1.3 MB 29.6 MB/s 
     |████████████████████████████████| 5.8 MB 33.7 MB/s 
     |████████████████████████████████| 12.8 MB 138 kB/s 
     |████████████████████████████████| 211 kB 6.5 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

from tensorflow import keras

In [ ]:
# load train and test dataset
def load_dataset():
# load dataset
  fashion_mnist = keras.datasets.fashion_mnist
  (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

  class_names = ["T-shirt/top", "Trouser", "Pullover","Dress", "Coat", "Sandal",
                 "Shirt","Sneaker", "Bag", "Ankle boot"]
	
  return train_images, train_labels, test_images, test_labels

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/fashion_mnist.h5')

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
#Define Quantization Model

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLay  (None, 28, 28)            3         
er)                                                              
_________________________________________________________________
quant_reshape (QuantizeWrap  (None, 28, 28, 1)         1         
per)                                                             
_________________________________________________________________
quant_conv2d_29 (QuantizeWr  (None, 28, 28, 32)        387       
apper)                                                           
_________________________________________________________________
quant_conv2d_30 (QuantizeWr  (None, 28, 28, 32)        9315      
apper)                                                           
_________________________________________________________________
quant_max_pooling2d_17 (Qua  (None, 14, 14, 32)       

In [ ]:
#train and evaluate model against baseline
train_images_subset = train_images[0:1000]  # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=5, validation_split=0.1)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1048: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 8s 3s/step - loss: 0.3141 - accuracy: 0.8978 - val_loss: 0.2738 - val_accuracy: 0.9200
Epoch 2/5
2/2 [==============================] - 6s 3s/step - loss: 0.2330 - accuracy: 0.9189 - val_loss: 0.1933 - val_accuracy: 0.9300
Epoch 3/5
2/2 [==============================] - 6s 3s/step - loss: 0.1819 - accuracy: 0.9322 - val_loss: 0.1674 - val_accuracy: 0.9500
Epoch 4/5
2/2 [==============================] - 6s 3s/step - loss: 0.1540 - accuracy: 0.9522 - val_loss: 0.1613 - val_accuracy: 0.9300
Epoch 5/5
2/2 [==============================] - 6s 3s/step - loss: 0.1273 - accuracy: 0.9589 - val_loss: 0.1503 - val_accuracy: 0.9500


In [ ]:
#Create Quantize model for TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpgn46rjd0/assets


INFO:tensorflow:Assets written to: /tmp/tmpgn46rjd0/assets


In [ ]:
#Evaluate the quantized model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1048: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Baseline test accuracy: 0.9326000213623047
Quant test accuracy: 0.9282000064849854


In [ ]:
#Check the size of the models
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: /tmp/tmphxesyuwh/assets


INFO:tensorflow:Assets written to: /tmp/tmphxesyuwh/assets


Float model in Mb: 1.6664085388183594
Quantized model in Mb: 0.43503570556640625


In [ ]:
q_aware_model.save('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/fashion_mnist_aware.h5')